In [0]:
import pandas as pd


In [2]:
from google.colab import files
uploaded = files.upload()

Saving cocktails_text.txt to cocktails_text.txt


In [4]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "cocktails_text.txt" with length 211680 bytes


In [0]:
data_path = "cocktails_text.txt"

In [0]:
with open(data_path, 'r') as f:
    lines = f.read().split('\n')

In [8]:
lines[666]

'Rum Old-Fashioned - Old-Fashioned Glass - 1 1/2 oz Light Rum - 1 oz 151-Proof Rum - 1 oz Simple Syrup - 1 oz Angostura Bitters -  Lemon twist - Stir in ice-filled old-fashioned glass. Add lemon twist.'

In [9]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

text = open(data_path, 'rb').read().decode(encoding='utf-8')

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


78 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
seq_length = 80
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

Instructions for updating:
Colocations handled automatically by placer.
'Gauguin - Old-Fashioned Glass - 2 oz Light Rum - 1 oz Passion Fruit Syrup - 1 oz '
'Lemon Juice - 1 oz Lime Juice - Combine ingredients with a cup of crushed ice in '
'blender and blend at low speed. Serve in old-fashioned glass. Top with a cherry.\n'
'Fort Lauderdale - Old-Fashioned Glass - 1 1/2 oz Light Rum - 1/2 oz Sweet Vermout'
'h - 1/4 oz Juice of Orange - 1/4 oz Juice of a Lime - Shake with ice and strain i'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 80), (64, 80)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.LSTM, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
  
  model.summary()

(64, 80, 78) # (batch_size, sequence_length, vocab_size)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           19968     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 78)            79950     
Total params: 5,346,894
Trainable params: 5,346,894
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 80, 78)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3552837


In [0]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
EPOCHS=3
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3
40/41 [============================>.] - ETA: 9s - loss: 3.3416 WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
41/41 [==============================] - 400s 10s/step - loss: 3.3335
Epoch 2/3
41/41 [==============================] - 394s 10s/step - loss: 2.4047
Epoch 3/3
41/41 [==============================] - 390s 10s/step - loss: 1.6455


In [25]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            19968     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 78)             79950     
Total params: 5,346,894
Trainable params: 5,346,894
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
print(generate_text(model, start_string=u"The Martin - "))

The Martin - pith oz and gith uice a Kithiin icle coruin glakee Frer. Tran.
Hith Colkal twith Hicebafled gllss..
Hihe Glemneed - 1 oz Grarerronde juit cere s- Sale Pocktal - O Cort and oz juice - 1 oz Solangeren Whicktipl Gladd sured s oz Gle Mpruin ingo in ere- fitneg and shice in to glinstelde erans witte and strila s Pocktail  Cocktail gl strriin - 2 oz Pondon Coumbowith isbodail glads Torminthe codkube ofdithienids Whicerane thior.
Cland with buiz  chan into intice gil ond grans into chibled cour ill -filed ollass wtishe. CCotain Flass - 1 oz Grin  wteri, - 34 oz OrbenYile and - Pock awe And straind into chillin gball ice outtin- Cocktail glass.
AadponEshice fillind dith ice aldss - 34 oz Angge - 3/4 oz Old Thice outt - Shaie glass  cocktail Glass Whyrektern, tlare.
Suin - Serleg Sockeis  Clain - Chilllil Glass - 3z oz PBich into cocktail - Shiland with bith.
Garbenith cocktail Glasss with bedke and jiict ice andintith icl anddostarrewpeingedd - 1/2 oz Grangor Voquymailnastir withi